# Sentiment analysis
1. Task type: NLP
2. Dataset: Tweets (Text type)
3. Usecases: Social Media management, Review Systems, News Analysis for Stock Markets

In [1]:
import pandas as pd
import numpy as np

In [2]:
# df = pd.read_csv('./data/train.csv', encoding='latin-1', names=['Target', 'TweetID', 'Date', 'No_Query', 'UserName', 'Data'])
# df.head()

Observation: 
1. Data is not utf-8 encoded that is why required to set correct encoding to read csv file.
2. Data is not having column names that is why provided it with column name.

In [3]:
# df.info()

As it is visible to me that Target and Data are only columns useful for me to train model for sentiment detection, I can drop other columns.

In [4]:
# df = df[['Target', 'Data']]
# df.head()

Now, I will try to remove words which usually does not contribute to sentiments like tags (@username in data in tweet) and urls. I will keep hashtags as of now to check if they make any effect on data or not. 

In [5]:
# df['Data'] = df['Data'].replace(r'http\S+', '', regex=True).replace(r'@\S+', '', regex=True)

In [6]:
# df.head(20)

Data info showing data is not having null values and datatypes are int64 or objects. Now I need to determine language of text for each statement as I want my model to get trained for english only. 

In [7]:
# from langdetect import detect

In [8]:
# from numpy import NaN


# for i in range(len(df)):
#     if df['Data'][i].isspace() == True:
#         df['Data'][i] = NaN

In [9]:
# df = df[df['Data'].noatna()]

In [10]:
# df = df.reset_index(drop=True)

In [11]:
# for i in range(len(df)):
#     try:
#         detect(df['Data'][i])
#     except:
#         print(i)

In [12]:
# import string
# for char in string.punctuation:
#     df['Data'] = df['Data'].replace(char, NaN, regex=False)

In [13]:
# df.head()

In [14]:
# df['ln']=[0]*len(df)
# print(df.head())
# for i in range(len(df)):
#     try:
#         x = detect(df['Data'][i])
#         df['ln'][i] = x
#     except:
#         df['ln'][i]=NaN

# for i in range(len(df)):
#     if df['ln'][i]=='en':
#         df['ln'][i]='en'
#     else:
#         df['ln'][i]=NaN

In [15]:
# df.head(20)

In [16]:
# df.to_csv('./data/trainModified.csv')

In [17]:
df = pd.read_csv('./data/trainModified.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Target,Data,ln
0,0,0,0,"- Awww, that's a bummer. You shoulda got Da...",en
1,1,1,0,is upset that he can't update his Facebook by ...,en
2,2,2,0,I dived many times for the ball. Managed to s...,en
3,3,3,0,my whole body feels itchy and like its on fire,en
4,4,4,0,"no, it's not behaving at all. i'm mad. why am...",en


In [18]:
df = df[df.ln == 'en']
len(df)

1469476

In [19]:
len(df[df.Target==0])/len(df[df.Target==4])

1.0135628243231949

It is now clearly visible that data is pretty cleaned and sample is almost of similar length so we can train model without thinking much about bias due to unbalanced data. Data length ratio of 1.01 suggests almost same length of both targets.

Reviews suggests that transformers are doing exceptional in language related tasks so I tried to find out how transformers can be applied for sentiment analysis. BERT is the model I was looking for. 
Implementing BERT with transformers and tensorflow.

In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading: 100%|██████████| 511M/511M [06:56<00:00, 1.29MB/s] 
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Downloading: 100%|██████████| 226k/226k [00:01<00:00, 129kB/s]  
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 27.8kB/s]


Tokenizer prepares text inputs for model. We are loading pretrained weights for BertTokenizer from pretrained model named "bert-base-cased".
Major usecase: question - answers, token classification etc.